<a href="https://colab.research.google.com/github/nikewinchester/NLPTextSummarization/blob/main/NLP_j_component.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import re
import string
from string import punctuation
import nltk

from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
nltk.download('punkt')
nltk.download("stopwords")
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from nltk import tokenize
from operator import itemgetter
import math
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data=pd.read_csv("https://raw.githubusercontent.com/nikewinchester/NLPTextSummarization/main/drugsComTest_raw.csv")

In [ ]:
df=pd.DataFrame(data=(data["drugName"][0:100],data["review"][0:100]))
df=df.T
for i in df.index:
  df['review'][i]=df['review'][i].replace("&#039;","'")
  df['review'][i]=df['review'][i].replace(",","")
  df['review'][i]=df['review'][i].replace("\\","")
  df['review'][i]=df['review'][i].replace("\"","")
df

,drugName,review
0,Mirtazapine,I've tried a few antidepressants over the year...
1,Mesalamine,My son has Crohn's disease and has done very w...
2,Bactrim,Quick reduction of symptoms
3,Contrave,Contrave combines drugs that were used for alc...
4,Cyclafem 1 / 35,I have been on this birth control for one cycl...
...,...,...
95,Humira,I really like this medication! It helps me wit...
96,"Insulin inhalation, rapid acting",Horrible
97,Acetaminophen / hydrocodone,I have been prescribed Vicodin 5/500s for over...
98,Phenazopyridine,I use Azo-Standard whenever I have issues with...


In [ ]:
df2 = df.groupby("drugName")
df_new = df2["review"].apply(list)
df_new = df_new.reset_index()
df_new.head()

,drugName,review
0,Acetaminophen / hydrocodone,[Was very beneficial when taken with a muscle ...
1,Actos,[Have been on Actos for almost a year gained 2...
2,Afrezza,[Since I was on the trials I put 6+ months but...
3,Aluminum chloride hexahydrate,[So in high school over ten years ago I starte...
4,Ambien,[Ditto on rebound sleepless when discontinued....


In [ ]:
def get_text_processing(text):
    stpword = stopwords.words('english')
    no_punctuation = [char for char in text if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return ' '.join([word for word in no_punctuation.split() if word.lower() not in stpword])

In [ ]:
df_new['rev'] = df_new['review'].apply(get_text_processing)
df_new.head()

,drugName,review,rev
0,Acetaminophen / hydrocodone,[Was very beneficial when taken with a muscle ...,beneficial taken muscle relaxer extreme pain n...
1,Actos,[Have been on Actos for almost a year gained 2...,Actos almost year gained 24 pounds swelling ha...
2,Afrezza,[Since I was on the trials I put 6+ months but...,Since trials put 6+ months back Afrezza 2 week...
3,Aluminum chloride hexahydrate,[So in high school over ten years ago I starte...,high school ten years ago started sweat really...
4,Ambien,[Ditto on rebound sleepless when discontinued....,Ditto rebound sleepless discontinued. done str...


In [ ]:
def Summarizer(test):
  df_new["review"][test]
  sentence = sent_tokenize(str(df_new["review"][test]))
  temp=df_new["rev"][test].split(".")
  all_words = [i for i in temp]
  model = Word2Vec([all_words], min_count=1,size= 300)
  sent_vector=[]
  temp=df_new["rev"][test].split(".")
  for i in temp:
      plus=0
      for j in i.split("."):
          plus+= model.wv[j]
      plus = plus/len(i.split("."))
      sent_vector.append(plus)
  n_clusters = 1
  kmeans = KMeans(n_clusters, init = 'k-means++', random_state = 42)
  y_kmeans = kmeans.fit_predict(sent_vector)
  my_list=[]
  for i in range(n_clusters):
      my_dict={}
    
      for j in range(len(y_kmeans)):
        
          if y_kmeans[j]==i:
              my_dict[j] =  distance.euclidean(kmeans.cluster_centers_[i],sent_vector[j])
      min_distance = min(my_dict.values())
      my_list.append(min(my_dict, key=my_dict.get))
 
  print(my_list)                          
  for i in sorted(my_list):
      print(sentence[i])

In [ ]:
test=13
df_new["review"][test]
Summarizer(5)

[0]
["This has been great for me.


In [ ]:
def check_sent(word, sentences): 
    final = [all([w in x for w in word]) for x in sentences] 
    sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
    return int(len(sent_len))

def get_top_n(dict_elem, n):
    result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
    return result.keys()

In [ ]:
def caltfidf(test):
  total_words = df_new["rev"][test].split()
  total_word_length = len(total_words)
  total_sentences = tokenize.sent_tokenize(df_new["rev"][test])
  total_sent_len = len(total_sentences)

  tf_score = {}
  for each_word in total_words:
      each_word = each_word.replace('.','')
      if each_word not in stop_words:
          if each_word in tf_score:
              tf_score[each_word] += 1
          else:
              tf_score[each_word] = 1

  # Dividing by total_word_length for each dictionary element
  tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())

  idf_score = {}
  for each_word in total_words:
      each_word = each_word.replace('.','')
      if each_word not in stop_words:
          if each_word in idf_score:
              idf_score[each_word] = check_sent(each_word, total_sentences)
          else:
              idf_score[each_word] = 1

  # Performing a log and divide
  idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())


  tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}

  tfidfw=list(get_top_n(tf_idf_score, 10))
  return tfidfw

In [ ]:
print(df_new['review'][test])

["I absolutely love this product and recommend to everyone. I know everyone's body is different so it is not for everyone but it is not the medicines fault. I have NO negative symptoms since I started this. I used to have heavy periods terrible cramps and headaches and my periods were super long. I now have super consistent 3-4 LIGHT periods no cramps or headaches. I have been on this pill for over a year now and have no desire to switch."]


In [ ]:
tfidfw=caltfidf(test)
print(tfidfw)

['periods', 'absolutely', 'love', 'product', 'recommend', 'know', "everyone's", 'body', 'different', 'medicines']
